In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path
from collections import defaultdict
import os
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from dotenv import load_dotenv
import json
import warnings
warnings.filterwarnings('ignore')

In [2]:
def numeric_nonNumeric_col(df):
    '''
    seperates the numeric and non-numeric columns.
    '''
    return [col for col in df.columns if isinstance(col, int)], [col for col in df.columns if isinstance(col, str)]
def interpolate_full_range(df):
    # Extract numeric column names
    numeric_cols, nonNumeric_cols = numeric_nonNumeric_col(df)

    # Function to fill the row with the first valid value
    def fill_initial_nans(series):
        first_valid_idx = series.first_valid_index()
        last_valid_idx = series.last_valid_index()
        if first_valid_idx is not None:  # Ensure there's a valid index
            series.loc[:first_valid_idx] = series[first_valid_idx]   # Fill initial NaNs
        if last_valid_idx is not None:
            series.loc[last_valid_idx:] = series[last_valid_idx]
        return series

    # Apply the function row-wise
    df[numeric_cols] = df[numeric_cols].apply(fill_initial_nans, axis=1)

    # Create full range of numeric columns from min to max
    full_range = np.arange(min(numeric_cols), max(numeric_cols) + 1)

    # Reindex DataFrame to include all missing columns
    df_numeric = df[numeric_cols].reindex(columns=full_range)

    # Interpolate missing values row-wise
    df_interpolated = df_numeric.interpolate(method='linear', axis=1)

    # Combine back with categorical columns
    return pd.concat([df_interpolated, df[nonNumeric_cols]], axis=1).reset_index(drop=True)


In [ ]:
# Load environment variables
load_dotenv()
FOLDER_PATH = os.getenv('FOLDER_PATH')
# Define the expected column names for the final concatenated DataFrame
CULTIVAR = "Cultivar" 
TREATMENT = "Treatment"
LOCATION = "Location"

# Select the interpolation method
interpolation_method = 'linear'#, 'akima', 'pchip', 'quadratic'

def is_date_column(col):
    # Function to check if column names are dates
    try:
        pd.to_datetime(col)  # Try converting the column name to a date
        return True
    except:
        return False

with open("config_MultiDataFiles.json", "r") as file:
    config = json.load(file)

# Load the Excel file

# Initialize an empty list to store filtered DataFrames
filtered_dfs = []

# Process each file mentioned in the config
for file_name, file_info in config.items():
    file_path = os.path.join(FOLDER_PATH, file_name)
    # if not file_path.exists():
    #     print(f"Warning: {file_name} not found in {FOLDER_PATH}")
    #     continue

    # Process each sheet
    for sheet_info in file_info['sheets']:
        # Extract sheet names, cultivars, and treatments
        sheet_name = sheet_info['sheet_name']
        location = sheet_info['location']
        cultivar_col, cultivar_name = list(sheet_info['cultivar'].items())[0]
        treatment_col, treatment_name = list(sheet_info['treatments'].items())[0]

        xls = pd.ExcelFile(file_path, engine="openpyxl")
        df = pd.read_excel(xls, sheet_name = sheet_name)

        # Ensure columns exist before filtering
        if cultivar_col not in df.columns or treatment_col not in df.columns:
            print(f"Warning: Missing required column(s) in {file_name} - sheet: {sheet_name}")
            continue
        
        # Rename the Dates to Days of Year (DOY)
        doy_column = [pd.to_datetime(col).dayofyear for col in df.columns if is_date_column(col)]
        df.rename(columns={col:pd.to_datetime(col).dayofyear for col in df.columns if is_date_column(col)}, inplace=True)

        # Apply filtering for the given cultivar & treatments
        filtered_df = df[df[cultivar_col].isin([cultivar_name]) & df[treatment_col].isin([treatment_name])]

        # Rename columns
        columns_to_select = [cultivar_col] + [treatment_col] + doy_column  # Ensure it's a flat list
        filtered_df = filtered_df[columns_to_select]
        filtered_df.rename(columns={cultivar_col: CULTIVAR, treatment_col: TREATMENT}, inplace=True)
        # Add the LOCATION column to the DataFrame
        filtered_df[LOCATION] = location

        # Remove rows with any NaN values
        filtered_df = filtered_df.dropna()

        # Reset index 
        filtered_df = filtered_df.reset_index(drop=True)

        # Store the filtered DataFrame
        filtered_dfs.append(filtered_df)

# Concatenate all filtered DataFrames
final_df = pd.concat(filtered_dfs, ignore_index=True, sort=True)



# Calculate cumulative/new daily buds {DayOfYear: [mean , std]} for all treatments & all locations
cumulative_daily = defaultdict(lambda: dict())
new_daily = defaultdict(lambda: dict())
final_df_treatment_loc_specific = defaultdict(lambda: dict())

all_treatments = final_df[TREATMENT].unique()
all_locations = final_df[LOCATION].unique()

for trtmnt in all_treatments:
    final_df_treatment_loc_specific[trtmnt]['all_loc'] = interpolate_full_range(final_df[final_df[TREATMENT].isin([trtmnt])])
    # Calculate mean & std for the number of buds per day
    numeric_columns, nonNumeric_cols = numeric_nonNumeric_col(final_df_treatment_loc_specific[trtmnt]['all_loc'])

    # For cumulative bud num (mean & std), for all locations
    cumulative_daily[trtmnt]['all_loc'] = pd.DataFrame({k:v for k,v in zip(['mean','std'],[[round(final_df_treatment_loc_specific[trtmnt]['all_loc'][day].mean(),2) for day in numeric_columns], [round(final_df_treatment_loc_specific[trtmnt]['all_loc'][day].std(ddof=0),2) for day in numeric_columns]])}, index=numeric_columns)
    
    # Taking the daily differences between the interpolated days
    diff_daily = final_df_treatment_loc_specific[trtmnt]['all_loc'][numeric_columns].copy()
    diff_daily.iloc[:, 1:] = diff_daily.iloc[:, 1:].values - diff_daily.iloc[:, :-1].values
    daily_diff = pd.concat([diff_daily,final_df_treatment_loc_specific[trtmnt]['all_loc'][nonNumeric_cols]], axis=1)

    # For new daily bud num (mean & std), for all locations 
    new_daily[trtmnt]['all_loc'] = pd.DataFrame({k:v for k,v in zip(['mean','std'],[[round(daily_diff[day].mean(),2) for day in numeric_columns], [round(daily_diff[day].std(ddof=0),2) for day in numeric_columns]])}, index=numeric_columns)
    
    # For new daily bud num (std), for all locations
    # daily_diff_all = final_df_treatment_loc_specific[trtmnt]['all_loc'][[col for col in final_df_treatment_loc_specific[trtmnt]['all_loc'].columns if type(col)==int]].dropna(axis=1).copy()
    # daily_diff_all.iloc[:, 1:] = daily_diff_all.diff(axis=1).iloc[:, 1:]  # Compute differences only for columns after the first
    # new_daily[trtmnt]['all_loc']['std'] = daily_diff_all.std(ddof=0).values
    
    for loc in all_locations:
        if final_df[final_df[TREATMENT].isin([trtmnt]) & final_df[LOCATION].isin([loc])].empty: # continue if for a given treatment the current location doesn't exist
            continue
        final_df_treatment_loc_specific[trtmnt][loc] = interpolate_full_range(final_df[final_df[TREATMENT].isin([trtmnt]) & final_df[LOCATION].isin([loc])])

        # For cumulative bud num (mean & std), per location
        cumulative_daily[trtmnt][loc] = pd.DataFrame({k:v for k,v in zip(['mean','std'],[[round(final_df_treatment_loc_specific[trtmnt][loc][day].mean(),2) for day in numeric_columns], [round(final_df_treatment_loc_specific[trtmnt][loc][day].std(ddof=0),2) for day in numeric_columns]])}, index=numeric_columns)
        
        # Taking the daily differences between the interpolated days
        diff_daily = final_df_treatment_loc_specific[trtmnt][loc][numeric_columns].copy()
        diff_daily.iloc[:, 1:] = diff_daily.iloc[:, 1:].values - diff_daily.iloc[:, :-1].values
        daily_diff = pd.concat([diff_daily,final_df_treatment_loc_specific[trtmnt][loc][nonNumeric_cols]], axis=1)
        # For new daily bud num (mean & std), for all locations 
        new_daily[trtmnt][loc] = pd.DataFrame({k:v for k,v in zip(['mean','std'],[[round(daily_diff[day].mean(),2) for day in numeric_columns], [round(daily_diff[day].std(ddof=0),2) for day in numeric_columns]])}, index=numeric_columns)
    

        # For new daily bud num (mean), per location
        # new_daily[trtmnt][loc] = cumulative_daily[trtmnt][loc].copy()
        # new_daily[trtmnt][loc]['mean'][1:] = new_daily[trtmnt][loc]['mean'].diff()[1:]
        # # For new daily bud num (std), per location
        # daily_diff_all = final_df_treatment_loc_specific[trtmnt][loc][[col for col in final_df_treatment_loc_specific[trtmnt][loc].columns if type(col)==int]].dropna(axis=1).copy()
        # daily_diff_all.iloc[:, 1:] = daily_diff_all.diff(axis=1).iloc[:, 1:]  # Compute differences only for columns after the first
        # new_daily[trtmnt][loc]['std'] = daily_diff_all.std(ddof=0).values
        
